In [2]:
import PIL
from PIL import Image

def load_image_file(file, mode='RGB'):
    im = PIL.Image.open(file)
    if mode:
        im = im.convert(mode)
    return np.array(im)

import dlib
import numpy as np
from PIL import ImageFile

def _raw_face_locations(img, number_of_times_to_upsample=1, model="hog"):
    """
    Returns an array of bounding boxes of human faces in a image

    :param img: An image (as a numpy array)
    :param number_of_times_to_upsample: How many times to upsample the image looking for faces. Higher numbers find smaller faces.
    :param model: Which face detection model to use. "hog" is less accurate but faster on CPUs. "cnn" is a more accurate
                  deep-learning model which is GPU/CUDA accelerated (if available). The default is "hog".
    :return: A list of dlib 'rect' objects of found face locations
    """
    if model == "cnn":
        return cnn_face_detector(img, number_of_times_to_upsample)
    else:
        return face_detector(img, number_of_times_to_upsample)
    
def face_locations(img, number_of_times_to_upsample=1, model="hog"):
    """
    Returns an array of bounding boxes of human faces in a image

    :param img: An image (as a numpy array)
    :param number_of_times_to_upsample: How many times to upsample the image looking for faces. Higher numbers find smaller faces.
    :param model: Which face detection model to use. "hog" is less accurate but faster on CPUs. "cnn" is a more accurate
                  deep-learning model which is GPU/CUDA accelerated (if available). The default is "hog".
    :return: A list of tuples of found face locations in css (top, right, bottom, left) order
    """
    if model == "cnn":
        return [_trim_css_to_bounds(_rect_to_css(face.rect), img.shape) for face in _raw_face_locations(img, number_of_times_to_upsample, "cnn")]
    else:
        return [_trim_css_to_bounds(_rect_to_css(face), img.shape) for face in _raw_face_locations(img, number_of_times_to_upsample, model)]

In [7]:
import face_recognition
import cv2
import numpy as np

# This is a demo of running face recognition on a video file and saving the results to a new video file.
#
# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Open the input movie file
input_movie = cv2.VideoCapture("sample2.mp4")
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

# Create an output movie file (make sure resolution/frame rate matches input video!)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_movie = cv2.VideoWriter('output.avi', fourcc, 29.97, (640, 360))

# Load some sample pictures and learn how to recognize them.
lmm_image = load_image_file("sample2_frame/frame 10.jpg")
lmm_face_encoding = face_recognition.face_encodings(lmm_image)

al_image = load_image_file("sample1_frame/frame 10.jpg")
al_face_encoding = face_recognition.face_encodings(al_image)

known_faces = [
    lmm_face_encoding,
    al_face_encoding
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
frame_number = 0

while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)

        # If you had more than 2 faces, you could make this logic a lot prettier
        # but I kept it simple for the demo
        name = None
        if match[0]:
            name = "sample1"
        elif match[1]:
            name = "sampe2"

        face_names.append(name)

    # Label the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if not name:
            continue

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    print("Writing frame {} / {}".format(frame_number, length))
    output_movie.write(frame)

# All done!
input_movie.release()
cv2.destroyAllWindows()

Writing frame 1 / 363
Writing frame 2 / 363
Writing frame 3 / 363
Writing frame 4 / 363
Writing frame 5 / 363
Writing frame 6 / 363
Writing frame 7 / 363
Writing frame 8 / 363
Writing frame 9 / 363
Writing frame 10 / 363
Writing frame 11 / 363
Writing frame 12 / 363
Writing frame 13 / 363
Writing frame 14 / 363
Writing frame 15 / 363
Writing frame 16 / 363
Writing frame 17 / 363
Writing frame 18 / 363
Writing frame 19 / 363
Writing frame 20 / 363
Writing frame 21 / 363
Writing frame 22 / 363
Writing frame 23 / 363
Writing frame 24 / 363
Writing frame 25 / 363
Writing frame 26 / 363
Writing frame 27 / 363
Writing frame 28 / 363
Writing frame 29 / 363
Writing frame 30 / 363
Writing frame 31 / 363
Writing frame 32 / 363
Writing frame 33 / 363
Writing frame 34 / 363
Writing frame 35 / 363
Writing frame 36 / 363
Writing frame 37 / 363
Writing frame 38 / 363
Writing frame 39 / 363
Writing frame 40 / 363
Writing frame 41 / 363
Writing frame 42 / 363
Writing frame 43 / 363
Writing frame 44 / 3

Writing frame 347 / 363
Writing frame 348 / 363
Writing frame 349 / 363
Writing frame 350 / 363
Writing frame 351 / 363
Writing frame 352 / 363
Writing frame 353 / 363
Writing frame 354 / 363
Writing frame 355 / 363
Writing frame 356 / 363
Writing frame 357 / 363
Writing frame 358 / 363
Writing frame 359 / 363
Writing frame 360 / 363
Writing frame 361 / 363
Writing frame 362 / 363
Writing frame 363 / 363
